In [1]:
import pandas as pd
import numpy as np

In [2]:
# Cargar los datos
# Notar que en un proyecto real, primero me familiarizo con los datos, desoués hago ML
data = pd.read_csv("../data/german.csv")

data.head()

# Aquí haríamos el EDA (Exploratory Data Analysis)

,Unnamed: 0,Status,Duration,CreditHistory,Purpose,CreditAmount,Savings,Employment,InstallmentRate,PersonalStatus,...,Property,Age,OtherInstallment,Housing,ExistingCredits,Job,LiablePeople,Telephone,ForeignWorker,Risk
0,0,A11,6,A34,A43,1169,A65,A75,4,A93,...,A121,67,A143,A152,2,A173,1,A192,A201,1
1,1,A12,48,A32,A43,5951,A61,A73,2,A92,...,A121,22,A143,A152,1,A173,1,A191,A201,2
2,2,A14,12,A34,A46,2096,A61,A74,2,A93,...,A121,49,A143,A152,1,A172,2,A191,A201,1
3,3,A11,42,A32,A42,7882,A61,A74,2,A93,...,A122,45,A143,A153,1,A173,2,A191,A201,1
4,4,A11,24,A33,A40,4870,A61,A73,3,A93,...,A124,53,A143,A153,2,A173,2,A191,A201,2


In [5]:
# Selecciono las variables de mi modelo
# Quiero predecir una cantidad (Credit Amount), por lo tanto
# este será un modelo Supervisado y de Regresión

# Yo creo que podemos predecir la cantidad de crédito con
# la duración (del prestamo) y la edad de las personas

X = data[['Duration','Age']]
y = data['CreditAmount']

In [6]:
# Vamos a dividir ahora los datos entre datos de entrenamiento y datos de testeo

# SIEMPRE SIEMPRE , pero SIEMPRE 
# vamos a hacer este train test split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

/Users/josemanuelbarrera/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [12]:
# La función train_test_split, en default
# divide los datos en 75% y 25% (entrenamiento y teseto respectivamente)
X_test.shape

(250, 2)

In [15]:
# Entrenamos el modelo ahora
from sklearn.linear_model import LinearRegression

modelo = LinearRegression()

modelo

LinearRegression()

In [17]:
modelo.fit(X_train, y_train)

LinearRegression()

In [30]:
# Este es el error cuadrático medio. Es la misma formula que la varianza
# y por lo tanto tiene los mismos beneficios y problemas.

resultados = pd.DataFrame({
    'predicciones': modelo.predict(X_test),
    'observaciones': y_test
})

resultados['diferencia'] = (resultados['predicciones'] - resultados['observaciones'])**2


resultados['diferencia'].mean()

np.sqrt(resultados['diferencia'].mean())

4202097.124114748

In [38]:
# En realidad, no tenemos que programar estas medidas del error. Las podemos importar
# directo de SKLearn y usarlas

from sklearn.metrics import mean_squared_error, r2_score

# Vamos a calcular el vector de predicciones
y_pred = modelo.predict(X_test)

# Este es el equivalente a la varianza, es decir
# entre más grande sea, mayor error tiene mi modelo.
# No puedo interpretar estos número
mse = mean_squared_error(y_test, y_pred)
mse

4202097.124114748

In [37]:
# Raíz del error cuadrático medio
# Al igual que la desviación estándar, este número
# está en las unidades de medida (euros)
# Por lo tanto, este número representa
# el margen de error de nuestro modelo en las unidades
# de medida de Y

rmse = np.sqrt(mse)
rmse

2049.9017352338496

In [40]:
# Esto es cuánta variabilidad de Y (cantidad de crédito) se explica
# con nuestro modelo.
# Solo el 36% de la cantidad prestada se explica
# con la edad y la duración del crédito.

# Es un modelo pobre
r2_score(y_test, y_pred)

0.36358141335462046

In [41]:
y.describe()

count     1000.000000
mean      3271.258000
std       2822.736876
min        250.000000
25%       1365.500000
50%       2319.500000
75%       3972.250000
max      18424.000000
Name: CreditAmount, dtype: float64

In [44]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

data = pd.read_csv("../data/german.csv")

X = data[['Duration','Age']]
y = data['CreditAmount']

X_train, X_test, y_train, y_test = train_test_split(X, y)

modelo = LinearRegression()
modelo.fit(X_train, y_train)

y_pred = modelo.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
score = r2_score(y_test, y_pred)

# Críticas al modelo

- Qué pasa con las otras variables que tenemos ? 
    - Podemos incorporar todas las variables que nosotros creemos que sirven para predecir el Credit Amount. Es nuestro trabajo como Data Scientist volver atrás y jugar con esta idea. 
    - Notar que para probar otras variables, tendremos que hacer transformaciones de datos. Hasta ahora sólo estamos usando variables númericas (cuantitativas). Para agregar otro tipo de variables (categorias) necesitariamos hacer transformaciones como one-hot-encoding (variables dummy, variables binarias).

- El tamaño de muestra, no siempre podemos mejorar la cantidad de observaciones
    - Básicamente más es mejor, siempre tener más de 50 observaciones.
    
- Probar con otro modelo. 
    - La regresión lineal es el modelo más "pobre" (básico) de ML. Aún así este tiempo no es desperdiciado. El modelo de regresión lineal lo utilizamos siempre como un baseline con el cual comparar.
    
- Ajustar los hiperparámetros del modelo
    - Revisar la documentación para ver qué parametros podemos modificar y que nos puedan ayudar a mejorar las métricas de error.
    
- Probar con otras transformaciones de datos (Normalización y Estandarización)
    - Existen otras herramientras para homologar los datos. Estas también pueden servir para mejorar el error de un modelo
    - También podemos probar con técnicas de reducción de dimensiones (PCA)

In [47]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

data = pd.read_csv("../data/german.csv")

X = data[['Duration','Age']]
y = data['CreditAmount']

X_train, X_test, y_train, y_test = train_test_split(X, y)

modelo = SVR()
modelo.fit(X_train, y_train)

y_pred = modelo.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
score = r2_score(y_test, y_pred)

In [48]:
mse

7584465.449458761

In [49]:
rmse

2753.9908223265306

In [50]:
score

-0.08012016911735409